# TF Lite and transfer learning

Implement all the ToDos.

You are working with the [Mendeley](https://data.mendeley.com/datasets/yz4v8tb3tp/5) dataset, which is modified version of the [LFWcrop](http://conradsanderson.id.au/lfwcrop/) dataset. The data is divided into two classes `smiling` and `non-smiling`. The samples for each class are located in a folder named after the class. There are aroung 600 images in each class.

In [1]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import collections
import h5py
import pathlib 
%matplotlib inline

In [2]:
# ToDo: load the data from disk. Be sure to keep the images associated to their class (folder name)
def loadDataset(directory):
    # keep link between images and labels (folder names). Use for example OrderedDict
    #result = collections.OrderedDict()
        
    #return result
    # keep link between images and labels (folder names). Use for example OrderedDict
    smiling_data_train = []
    non_smiling_data_train = []
    smiling_data_test = []
    non_smiling_data_test = []
    for x in os.walk(directory):
        if x[0] != directory:
            cur = x[0]
            dir_name = cur.split(directory,1)[1]
            data = os.listdir(cur)
            for sample in data:
                index = data.index(sample)
                img_path = cur + '/' + sample
                x = keras.preprocessing.image.load_img(img_path, target_size=(96,96))
                x = keras.preprocessing.image.img_to_array(x)
                x = np.expand_dims(x, axis=0)
                if dir_name == "smiling":
                    if index % 5 == 0:
                        smiling_data_test.append(x)
                    else:
                        smiling_data_train.append(x)
                else:
                    if index % 5 == 0:
                        non_smiling_data_test.append(x)
                    else:
                        non_smiling_data_train.append(x)
        
    return np.squeeze(smiling_data_train), np.squeeze(non_smiling_data_train), np.squeeze(smiling_data_test), np.squeeze(non_smiling_data_test)

In [3]:
smiling_data_train, non_smiling_data_train, smiling_data_test, non_smiling_data_test = loadDataset("resources/dataset/")

In [4]:
print(smiling_data_train.shape)
print(non_smiling_data_train.shape) 
print(smiling_data_test.shape)
print(non_smiling_data_test.shape)

(480, 96, 96, 3)
(482, 96, 96, 3)
(120, 96, 96, 3)
(121, 96, 96, 3)


#### Generate Test and Train sets and according labels

In [5]:
#data = loadDataset("resources/dataset/")

# ToDo: split the dataset into subsets for training and testing.
#X_train = 
#X_test = 


#y_train =                  
#y_test = 
from tensorflow.keras.utils import to_categorical

#train
labels_smiling = np.ones(smiling_data_train.shape[0])
labels_non_smiling = np.zeros(non_smiling_data_train.shape[0])
y_train = np.append(labels_smiling, labels_non_smiling, axis=0)
print(y_train.shape)
y_binary = to_categorical(y_train)
print(y_binary[-1])
print(y_binary.shape)
X_train = np.append(smiling_data_train, non_smiling_data_train, axis=0)
print(X_train.shape)
#test
labels_smiling = np.ones(smiling_data_test.shape[0])
labels_non_smiling = np.zeros(non_smiling_data_test.shape[0])
y_test = np.append(labels_smiling, labels_non_smiling, axis=0)
y_binary_test = to_categorical(y_test)
X_test = np.append(smiling_data_test, non_smiling_data_test, axis=0)
print(y_binary_test.shape)
print(X_test.shape)

(962,)
[1. 0.]
(962, 2)
(962, 96, 96, 3)
(241, 2)
(241, 96, 96, 3)


In [6]:
# ToDo: Load a pretrained Network using keras. Choose from https://keras.io/applications/
# ALTERNATIVELY: Define your own network using the keras sequential API: https://keras.io/getting-started/sequential-model-guide/
base_model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(96,96,3), alpha=0.5,  include_top=False)
base_model.trainable = False
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 16)   432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 48, 48, 16)   64          Conv1[0][0]                      
_____________________________________

In [7]:
image_batch = X_train[1:33]
print(image_batch.shape)
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 96, 96, 3)
(32, 3, 3, 1280)


In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [9]:
prediction_layer = keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 2)


In [15]:
# ToDo: Append new output layers to the loaded model using the keras sequential API: https://keras.io/getting-started/sequential-model-guide/


model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

base_learning_rate = 0.0001

#model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_0.50_96 (Model)  (None, 3, 3, 1280)        706224    
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 708,786
Trainable params: 2,562
Non-trainable params: 706,224
_________________________________________________________________


In [17]:
# ToDo define training checkpointing

checkpoint_path = "weights-improvement-{epoch:02d}.hdf5"
checkpoint_dir = './checkpoints'
filepath = checkpoint_dir + '/' + checkpoint_path

# ToDo: create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath, verbose=1, period=5)

In [21]:
!cp ./checkpoints/weights-improvement-20.hdf5 ./83pc_acc.hdf5

In [25]:
model.load_weights('./83pc_acc.hdf5')
loss, acc = model.evaluate(X_test, y_binary_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
model.save(fileName)

241/241 [==============================] - 0s 238us/sample - loss: 5.3393 - acc: 0.8340
Restored model, accuracy: 83.40%


In [23]:
# ToDo: Train your model and save it as HDF5 file
# again, use https://keras.io/models/model/ 

fileName = "model.h5"

model.fit(X_train, y_binary, callbacks=[cp_callback], epochs=35, batch_size=64, verbose=0)
model.evaluate(X_test, y_binary_test, batch_size=10)

model.save(fileName)


Epoch 00005: saving model to ./checkpoints/weights-improvement-05.hdf5

Epoch 00010: saving model to ./checkpoints/weights-improvement-10.hdf5

Epoch 00015: saving model to ./checkpoints/weights-improvement-15.hdf5

Epoch 00020: saving model to ./checkpoints/weights-improvement-20.hdf5

Epoch 00025: saving model to ./checkpoints/weights-improvement-25.hdf5

Epoch 00030: saving model to ./checkpoints/weights-improvement-30.hdf5

Epoch 00035: saving model to ./checkpoints/weights-improvement-35.hdf5
241/241 [==============================] - 0s 540us/sample - loss: 7.7092 - acc: 0.5021


In [ ]:
# Run this command for detailed documentation on the attributes of tflite-convert.
#help(tf.lite.TFLiteConverter)

In [26]:
# ToDo: Convert to TensorFlow Lite model and save.
converter = tf.lite.TFLiteConverter.from_keras_model_file(fileName)
tflite_model = converter.convert()
open("graph.tflite", "wb").write(tflite_model)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 262 variables.
INFO:tensorflow:Converted 262 variables to const ops.


2748752